In [4]:
import numpy as np
import os
from sklearn.ensemble import RandomForestClassifier
import csv
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from random import random

In [5]:
def RandomForest_fusion (lFilesPred, sFileLabels, n_estimators):
    
    # Read file labels (true labels)
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dID = {rows[0]:rows[1] for rows in reader} #participant ID
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dOnOff= {rows[0]:rows[2] for rows in reader} #on-off label
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dDys={rows[0]:rows[3] for rows in reader} #dyskinesia label
    with open(sFileLabels, mode='r') as infile:
        reader = csv.reader(infile)
        dTrem={rows[0]:rows[4] for rows in reader} #tremor label
#mydict = dict((rows[0],rows[1]) for rows in reader)
    
    # Training-testing data
    iNumFiles=len(lFilesPred)

#mPredictions=[] #np.zeros((1,iNumFiles))
#mPredictions=np.asarray(mPredictions)
    vPredIter=np.zeros((1,iNumFiles))
    vLabels=[] #true label
    vParID=[] # participant ID
    lDicts=[] 

    for sFilePred in lFilesPred:
        with open(sFilePred, mode='r') as infile:
            reader = csv.reader(infile)
            dPred = {rows[0]:rows[1] for rows in reader} #Prediction from the different classifiers (ivec-svr, boost...)
        lDicts.append(dPred)
      
    #label selection
    if sTypeLabel=='on_off':
        dLabels=dOnOff
    elif sTypeLabel=='tremor':
        dLabels=dTrem
    elif sTypeLabel=='dyskinesia':
        dLabels=dDys
    else:
        print('sTypeLabel undefined')
    
    #creation of the matrix containing prediction from all classifiers
    bEnter=1
    for k in lDicts[0]: #first dictionary will be the lead
        if k!='measurement_id':
            #print(k)
            #print(dID[k])
            if dLabels[k]!='NA':
                vLabels.append(float(np.asarray(dLabels[k]))) #true labels
                vParID.append(float(np.asarray(dID[k]))) #participant ID
                vPredIter[0,0]=float(np.asarray(lDicts[0][k])) #first predicted value
                for j in range(1, iNumFiles):
                    fPred=lDicts[j].get(k)
                if fPred:
                    vPredIter[0,j]=float(np.asarray(fPred))
                else:
                    print(['Unkwnown key:' + k])
                    vPredIter[0,j]=float(np.asarray(lDicts[0][k]))
                    
            if bEnter==1:
                mPredictions=vPredIter
                bEnter=0
            else:
                mPredictions=np.append(mPredictions,vPredIter,axis=0)
         
        
    # Random forest training - regression
    clf=RandomForestRegressor(n_estimators)#, max_depth=None, min_samples_split=2, random_state=0)
#clf=RandomForestClassifier(n_estimators)
#clf = clf.fit(X, Y)
#clf.score(X_test, y_test)

    scores = cross_val_score(clf, mPredictions, vLabels, cv=15)
    scores.mean()

    clf=clf.fit(mPredictions, vLabels)
    vNewPred=clf.predict(mPredictions)

    # we will include the testing data here
    return (vNewPred, vParID, vLabels)
#print(mPredictions)

In [6]:
sFilePred1='/export/b15/nchen/BeatPD/cispd.kfold_prediction_on_off.csv'
sFilePred2='/export/c08/lmorove1/kaldi/egs/beatPDivec/on_off_noinact_auto30/exp/ivec_450/resiVecSVR_Fold_all/objs_400_kernel_linear_c_20.0_eps_20.0.csv'
sTypeLabel='on_off'
n_estimators=10
sFileLabels='/export/b18/sjoshi/BeatPD/data/BeatPD/cis-pd.data_labels/CIS-PD_Training_Data_IDs_Labels.csv'

lFilesPred=[sFilePred1,sFilePred2];

vPredictions, vParID, vTrueLabels =RandomForest_fusion (lFilesPred, sFileLabels, n_estimators)



In [7]:
print(len(vPredictions))
print(len(vParID))
print(len(vTrueLabels))

1767
1767
1767


In [17]:
%load_ext autoreload
%autoreload 2
from get_final_scores_accuracy import final_score

def get_final_score(vPredictions, vParID, vTrueLabels):
    """
    Compute the final score from the challenge and print the result
    
    Keyword arguments: 
    - vPredictions: Numpy array containing the predictions 
    - VParID: list containing the subject_id 
    - vTrueLabels: list containing the true labels 
    """
    mse_per_subjectId = []
    nb_files_per_subjectId = []
    
    for subject_id in np.unique(vParID):
        print('--- SUBJECT ID ', subject_id, '---')
        
        vSubjectId = (vParID == subject_id)

        vPredictions_subjectId = vPredictions[vSubjectId]

        vTrueLabels_subjectId = np.array(vTrueLabels)[vSubjectId]
        mse_per_subjectId.append(mean_squared_error(vTrueLabels_subjectId, vPredictions_subjectId))
        nb_files_per_subjectId.append(len(vPredictions_subjectId))
        
        print('MSE : ', mean_squared_error(vTrueLabels_subjectId, vPredictions_subjectId))
    
    print('--- FUSION ---')
    final_score(mse_per_subjectId, nb_files_per_subjectId)
    
get_final_score(vPredictions, vParID, vTrueLabels)

print('Overall MSE : ', mean_squared_error(vTrueLabels, vPredictions))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
--- SUBJECT ID  1004.0 ---
MSE :  0.4610998281895649
--- SUBJECT ID  1006.0 ---
MSE :  0.19306379175913427
--- SUBJECT ID  1007.0 ---
MSE :  0.8666823156727909
--- SUBJECT ID  1019.0 ---
MSE :  1.010563088689475
--- SUBJECT ID  1020.0 ---
MSE :  0.40461828984025794
--- SUBJECT ID  1023.0 ---
MSE :  0.6929249667544984
--- SUBJECT ID  1032.0 ---
MSE :  0.40669200489130414
--- SUBJECT ID  1034.0 ---
MSE :  0.5383792572376871
--- SUBJECT ID  1038.0 ---
MSE :  1.3713963422495905
--- SUBJECT ID  1039.0 ---
MSE :  0.5355143172089107
--- SUBJECT ID  1043.0 ---
MSE :  0.8490159953275616
--- SUBJECT ID  1044.0 ---
MSE :  0.14849072129839166
--- SUBJECT ID  1048.0 ---
MSE :  0.25104904974505937
--- SUBJECT ID  1049.0 ---
MSE :  0.40552148664418974
--- SUBJECT ID  1051.0 ---
MSE :  0.8051466658687598
--- FUSION ---
Final score :  0.6288013926621604
Overall MSE :  0.6678261371110267
